In [1]:
from src.db.db_utils import db_utils
from src.db.controller import init_async_db
from src.config import ETHER_protocols
import pandas as pd
import numpy as np
import plotly.express as px
import matplotlib as mpl

mpl.rcParams['figure.figsize'] = (10, 8)
mpl.rcParams['axes.grid'] = False

In [2]:

swaps_table, pools_table, future_table = await init_async_db(False)
db_utils.set_tables(swaps_table, pools_table, future_table)

swaps_table = db_utils.get_swaps()

pool = '0x328dfd0139e26cb0fef7b0742b49b0fe4325f821'

swaps_rows = await swaps_table.get_all_rows_by_criteria({'pool_contract': pool})
print(len(swaps_rows))


0


In [3]:

df = pd.DataFrame([t.__dict__ for t in swaps_rows])

In [4]:
df.head()

""


In [5]:
df['timestamp'] = pd.to_datetime(df['timestamp'], unit='s')

KeyError: 'timestamp'

In [ ]:
df = df.set_index('timestamp').resample('Min').mean().reset_index()

In [ ]:
df.head()

In [ ]:
df.shape

In [ ]:
fig = px.line(df.reset_index(), x='timestamp', y='price', title='Price Changes')
#slider
fig.update_xaxes(
    rangeslider_visible = True,
)
fig.show()

In [ ]:
from prophet import Prophet
taxi_df = df.reset_index()[['timestamp', 'price']].rename({'timestamp':'ds', 'price':'y'}, axis='columns')

In [ ]:
train = taxi_df
# test = taxi_df[(taxi_df['ds'] > '2022-03-28')]

In [ ]:
# train['cap'] = 9 * 10**13
# train['floor'] = 0

m = Prophet(changepoint_range=1)#(growth='logistic')
m.fit(train)

In [ ]:
future = m.make_future_dataframe(periods=30, freq='Min')
# future['cap'] = 10**13
# future['floor'] = 0

In [ ]:
forecast = m.predict(future)
forecast[['ds','yhat','yhat_lower','yhat_upper']].tail()

In [ ]:
forecast.shape

In [ ]:
result = pd.concat([taxi_df.set_index('ds')['y'], forecast.set_index('ds')[['yhat','yhat_lower','yhat_upper']]], axis=1)
fig1 = m.plot(forecast, figsize=(16, 8), xlabel='timestamp', ylabel='price')
# fig1.savefig('weth_usdc.png')

In [ ]:
result['error'] = result['y'] - result['yhat']
result['uncertainty'] = result['yhat_upper'] - result['yhat_lower']

In [ ]:
result['anomaly'] = result.apply(lambda x: 'Yes' if(np.abs(x['error']) > 1.5*x['uncertainty']) else 'No', axis = 1)

In [ ]:
fig = px.scatter(result.reset_index(), x='ds', y='y', color='anomaly', title='NYC Taxi Demand')
# fig.add_scatter(result, x='ds', y='yhat', color='green', title='real')
#slider
# fig.update_xaxes(
#     rangeslider_visible = True,
# )
fig.show()